### Testing

In [4]:
%cd ..

/Users/youngoh/Desktop/flood_urop_repo/mml_flood


/Users/youngoh/Desktop/flood_urop_repo/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
import pandas as pd
import utils
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics


In [258]:

def baseline(y_test, results):
	y_base = np.zeros(len(y_test)) 
	print('Dumb baseline AUC', metrics.roc_auc_score(y_test,y_base))
	results['base1']=utils.get_scores_clf(y_test,y_base)

def stats_only(x_train, y_train, x_test, y_test, results):
	y_pred, y_pred_prob = utils.run_xgb(x_train.iloc[:, :21], np.ravel(y_train), x_test.iloc[:, :21])
	results['stats only'] = utils.get_scores_clf(y_test, y_pred_prob)

def nlp_only(x_train, y_train, x_test, y_test, results):
	y_pred, y_pred_prob = utils.run_xgb(x_train.iloc[:, 25:153], np.ravel(y_train), x_test.iloc[:, 25:153])
	results['nlp only'] = utils.get_scores_clf(y_test, y_pred_prob)

def weather_only(x_train, y_train, x_test, y_test, results):
	y_pred, y_pred_prob = utils.run_xgb(x_train.iloc[:, 153:], np.ravel(y_train), x_test.iloc[:, 153:])
	results['weather only'] = utils.get_scores_clf(y_test, y_pred_prob)
 
def stats_nlp(x_train, y_train, x_test, y_test, results):
	y_pred, y_pred_prob = utils.run_xgb(x_train.iloc[:, :153], np.ravel(y_train), x_test.iloc[:, :153])
	results['stats nlp'] = utils.get_scores_clf(y_test, y_pred_prob)

def weather_nlp(x_train, y_train, x_test, y_test, results):
	y_pred, y_pred_prob = utils.run_xgb(x_train.iloc[:, 25:], np.ravel(y_train), x_test.iloc[:, 25:])
	results['weather nlp'] = utils.get_scores_clf(y_test, y_pred_prob)
 
def stats_weather(x_train, y_train, x_test, y_test, results):
	y_pred, y_pred_prob = utils.run_xgb(x_train.drop(columns=x_train.columns[25:25+128]), np.ravel(y_train), x_test.drop(columns=x_train.columns[25:25+128]))
	results['stats weather'] = utils.get_scores_clf(y_test, y_pred_prob)

def all_vars(x_train, y_train, x_test, y_test, results):
	y_pred, y_pred_prob = utils.run_xgb(x_train, np.ravel(y_train), x_test)
	results['all vars'] = utils.get_scores_clf(y_test, y_pred_prob)

In [6]:

def baseline(y_test, results):
	y_base = np.zeros(len(y_test)) 
	print('Dumb baseline AUC', metrics.roc_auc_score(y_test,y_base))
	results['base1']=utils.get_scores_clf(y_test,y_base)

def stats_only(x_train, y_train, x_test, y_test, results):
	x_train, x_test = x_train.filter(regex='stat'), x_test.filter(regex='stat')
	y_pred, y_pred_prob = utils.run_xgb(x_train, np.ravel(y_train), x_test)
	results['stats only'] = utils.get_scores_clf(y_test, y_pred_prob)

def nlp_only(x_train, y_train, x_test, y_test, results):
	x_train, x_test = x_train.filter(regex='nlp'), x_test.filter(regex='nlp')
	y_pred, y_pred_prob = utils.run_xgb(x_train, np.ravel(y_train), x_test)
	results['nlp only'] = utils.get_scores_clf(y_test, y_pred_prob)

def weather_only(x_train, y_train, x_test, y_test, results):
	x_train, x_test = x_train.filter(regex='era'), x_test.filter(regex='era')
	y_pred, y_pred_prob = utils.run_xgb(x_train, np.ravel(y_train), x_test)
	results['weather only'] = utils.get_scores_clf(y_test, y_pred_prob)
 
def stats_nlp(x_train, y_train, x_test, y_test, results):
	x_train, x_test = x_train.filter(regex='stat|nlp'), x_test.filter(regex='stat|nlp')
	y_pred, y_pred_prob = utils.run_xgb(x_train, np.ravel(y_train), x_test)
	results['stats nlp'] = utils.get_scores_clf(y_test, y_pred_prob)

def weather_nlp(x_train, y_train, x_test, y_test, results):
	x_train, x_test = x_train.filter(regex='weather|nlp'), x_test.filter(regex='weather|nlp')
	y_pred, y_pred_prob = utils.run_xgb(x_train, np.ravel(y_train), x_test)
	results['weather nlp'] = utils.get_scores_clf(y_test, y_pred_prob)
 
def stats_weather(x_train, y_train, x_test, y_test, results):
	x_train, x_test = x_train.filter(regex='stat|era'), x_test.filter(regex='stat|era')
	y_pred, y_pred_prob = utils.run_xgb(x_train, np.ravel(y_train), x_test)
	results['stats weather'] = utils.get_scores_clf(y_test, y_pred_prob)

def all_vars(x_train, y_train, x_test, y_test, results):
	y_pred, y_pred_prob = utils.run_xgb(x_train, np.ravel(y_train), x_test)
	results['all vars'] = utils.get_scores_clf(y_test, y_pred_prob)

# N = 1

In [63]:
# Load the dataset
xy_df_sub1 = pd.read_csv('data/xy_df_renamed/xy_df_sub1_full.csv')
x = xy_df_sub1.drop(xy_df_sub1.filter(regex='target').columns, axis=1)  # Drop target columns
# x = x.drop(x.filter(regex='flood'), axis=1)
x = x.select_dtypes(['number'])  # Keep only numerical columns
y = xy_df_sub1.filter(regex='target')  # Keep only target columns

column_index = x.columns.get_loc('year')
x = x.iloc[:, column_index+1:]

# Get the 'year' column
years = xy_df_sub1['year']

# Determine the split based on years
min_year, max_year = years.min(), years.max()
unique_years = np.sort(years.unique())
split_year = unique_years[int(len(unique_years) * 0.8)]  # 80% split point

# Split data chronologically
train_mask = years <= split_year
test_mask = years > split_year

x_train1, x_test1 = x[train_mask], x[test_mask]
y_train1, y_test1 = y[train_mask], y[test_mask]

print(x_train1.shape, x_test1.shape, y_train1.shape, y_test1.shape)


(39840, 539) (9130, 539) (39840, 1) (9130, 1)


In [64]:
x_train1.filter(regex='stat').columns

Index(['stat_flood_amt', 'stat_storm_amt', 'stat_earthquake_amt',
       'stat_extreme temperature _amt', 'stat_landslide_amt',
       'stat_volcanic activity_amt', 'stat_drought_amt',
       'stat_mass movement (dry)_amt', 'stat_flood_ct', 'stat_storm_ct',
       'stat_earthquake_ct', 'stat_extreme temperature _ct',
       'stat_landslide_ct', 'stat_volcanic activity_ct', 'stat_drought_ct',
       'stat_mass movement (dry)_ct', 'stat_flood_bin', 'stat_storm_bin',
       'stat_earthquake_bin', 'stat_extreme temperature _bin',
       'stat_landslide_bin', 'stat_volcanic activity_bin', 'stat_drought_bin',
       'stat_mass movement (dry)_bin', 'stat_lat', 'stat_lon'],
      dtype='object')

In [65]:
results = {}
y_pred, y_pred_prob = utils.run_xgb(x_train1.filter(regex='stat'), y_train1, x_test1.filter(regex='stat'))
results['stats only chrono split'] = utils.get_scores_clf(y_test1, y_pred_prob)

running xgb...
[10, 20, 65]
Train AUC:  0.657076746401628
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.5375003577155317 0.5
auc, f1, accu, accu_bl, precision, recall=  0.5491766259491582 0.5130012001358629 0.8076670317634173 0.5415139367070698 0.1192665256149098 0.20307692307692307
[[7176  979]
 [ 777  198]]


In [18]:
results1 = {}

In [55]:
baseline(y_test1, results1)

Dumb baseline AUC 0.5
maximum f1 score, thres 0.47179635522129015 0.4
auc, f1, accu, accu_bl, precision, recall=  0.5 0.47179635522129015 0.8932092004381161 0.5 0.10679079956188389 0.0
[[8155    0]
 [ 975    0]]


In [56]:
stats_only(x_train1, y_train1, x_test1, y_test1, results1)

running xgb...
[10, 20, 65]
Train AUC:  0.657076746401628
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.5375003577155317 0.5
auc, f1, accu, accu_bl, precision, recall=  0.5491766259491582 0.5130012001358629 0.8076670317634173 0.5415139367070698 0.1192665256149098 0.20307692307692307
[[7176  979]
 [ 777  198]]


In [57]:
nlp_only(x_train1, y_train1, x_test1, y_test1, results1)

running xgb...
[10, 20, 65]
Train AUC:  0.6874800067310671
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 20}
maximum f1 score, thres 0.5269829350145094 0.6
auc, f1, accu, accu_bl, precision, recall=  0.4956670282507193 0.5233710143560297 0.8050383351588171 0.5296577528337185 0.11481824734836782 0.1794871794871795
[[7175  980]
 [ 800  175]]


In [58]:
weather_only(x_train1, y_train1, x_test1, y_test1, results1)

running xgb...
[10, 20, 65]
Train AUC:  0.895953583647672
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.508646254471562 0.4
auc, f1, accu, accu_bl, precision, recall=  0.5099862598060023 0.47164351851851855 0.7772179627601314 0.5131814680312534 0.10969025703390922 0.17743589743589744
[[6923 1232]
 [ 802  173]]


In [59]:
stats_nlp(x_train1, y_train1, x_test1, y_test1, results1)

running xgb...
[10, 20, 65]
Train AUC:  0.7068831097697108
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.5423238542423087 0.5
auc, f1, accu, accu_bl, precision, recall=  0.5136236696065023 0.5102763973771379 0.8286966046002191 0.5415466364822588 0.12047331216199615 0.1764102564102564
[[7394  761]
 [ 803  172]]


In [60]:
weather_nlp(x_train1, y_train1, x_test1, y_test1, results1)

running xgb...
[10, 20, 65]
Train AUC:  0.6874800067310671
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 20}
maximum f1 score, thres 0.5269829350145094 0.6
auc, f1, accu, accu_bl, precision, recall=  0.4956670282507193 0.5233710143560297 0.8050383351588171 0.5296577528337185 0.11481824734836782 0.1794871794871795
[[7175  980]
 [ 800  175]]


In [61]:
stats_weather(x_train1, y_train1, x_test1, y_test1, results1)

running xgb...
[10, 20, 65]
Train AUC:  0.8627358216483783
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.5331306369170259 0.5
auc, f1, accu, accu_bl, precision, recall=  0.5267770284079297 0.4935887915489241 0.831763417305586 0.5315241553868163 0.11642976599007052 0.14974358974358976
[[7448  707]
 [ 829  146]]


In [62]:
all_vars(x_train1, y_train1, x_test1, y_test1, results1)

running xgb...
[10, 20, 65]
Train AUC:  0.8659603154958664
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.5357186321041743 0.5
auc, f1, accu, accu_bl, precision, recall=  0.5313803392601676 0.4976199255465445 0.8328587075575027 0.5339433099089752 0.11747886420489878 0.15384615384615385
[[7454  701]
 [ 825  150]]


# N = 2

In [67]:
xy_df_sub2 = pd.read_csv('data/xy_df_renamed/xy_df_sub2_full.csv')
x = xy_df_sub2.drop(xy_df_sub2.filter(regex='target').columns, axis=1)  # Drop target columns
# x = x.drop(x.filter(regex='flood'), axis=1)
x = x.select_dtypes(['number'])  # Keep only numerical columns
y = xy_df_sub2.filter(regex='target')  # Keep only target columns

column_index = x.columns.get_loc('year')
x = x.iloc[:, column_index+1:]

# Get the 'year' column
years = xy_df_sub2['year']

# Determine the split based on years
min_year, max_year = years.min(), years.max()
unique_years = np.sort(years.unique())
split_year = unique_years[int(len(unique_years) * 0.8)]  # 80% split point

# Split data chronologically
train_mask = years <= split_year
test_mask = years > split_year

x_train2, x_test2 = x[train_mask], x[test_mask]
y_train2, y_test2 = y[train_mask], y[test_mask]

print(x_train2.shape, x_test2.shape, y_train2.shape, y_test2.shape)


(39840, 539) (9130, 539) (39840, 1) (9130, 1)


In [68]:
results2 = {}

baseline(y_test2, results2)

Dumb baseline AUC 0.5
maximum f1 score, thres 0.4487047883581909 0.4
auc, f1, accu, accu_bl, precision, recall=  0.5 0.4487047883581909 0.8139101861993429 0.5 0.18608981380065717 0.0
[[7431    0]
 [1699    0]]


In [69]:
stats_only(x_train2, y_train2, x_test2, y_test2, results2)

running xgb...
[10, 20, 65]
Train AUC:  0.6552563371295496
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 20}
maximum f1 score, thres 0.5378351895401636 0.7
auc, f1, accu, accu_bl, precision, recall=  0.5252114232179924 0.5378351895401636 0.7001095290251916 0.5415496493579661 0.2025217606360067 0.28899352560329605
[[5901 1530]
 [1208  491]]


In [70]:
nlp_only(x_train2, y_train2, x_test2, y_test2, results2)

running xgb...
[10, 20, 65]
Train AUC:  0.681419150789009
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.52742323015734 0.6
auc, f1, accu, accu_bl, precision, recall=  0.48476194843848475 0.5053903245029525 0.7499452354874042 0.52676362776904 0.1969627264505936 0.17127722189523248
[[6556  875]
 [1408  291]]


In [71]:
weather_only(x_train2, y_train2, x_test2, y_test2, results2)

running xgb...
[10, 20, 65]
Train AUC:  0.82386749603097
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.506001317587967 0.6
auc, f1, accu, accu_bl, precision, recall=  0.5139878999805865 0.45936593098667605 0.7600219058050384 0.5113884306148249 0.1902001199040897 0.1153619776339023
[[6743  688]
 [1503  196]]


In [72]:
weather_nlp(x_train2, y_train2, x_test2, y_test2, results2)

running xgb...
[10, 20, 65]
Train AUC:  0.681419150789009
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.52742323015734 0.6
auc, f1, accu, accu_bl, precision, recall=  0.48476194843848475 0.5053903245029525 0.7499452354874042 0.52676362776904 0.1969627264505936 0.17127722189523248
[[6556  875]
 [1408  291]]


In [73]:
stats_nlp(x_train2, y_train2, x_test2, y_test2, results2)

running xgb...
[10, 20, 65]
Train AUC:  0.6970444066369286
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.5410778010484428 0.6
auc, f1, accu, accu_bl, precision, recall=  0.49987394327994117 0.5259437307505268 0.7349397590361446 0.5395649787739176 0.20278254275917515 0.22836962919364331
[[6322 1109]
 [1311  388]]


In [74]:
stats_weather(x_train2, y_train2, x_test2, y_test2, results2)

running xgb...
[10, 20, 65]
Train AUC:  0.8119334373944899
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.5330339295336427 0.6
auc, f1, accu, accu_bl, precision, recall=  0.5221162020389427 0.5209816668709053 0.7421686746987952 0.5315205561164676 0.19902407915325024 0.19599764567392583
[[6443  988]
 [1366  333]]


In [75]:
all_vars(x_train2, y_train2, x_test2, y_test2, results2)

running xgb...
[10, 20, 65]
Train AUC:  0.81315465717013
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.5343750321670836 0.6
auc, f1, accu, accu_bl, precision, recall=  0.5224726696912358 0.5175477739100136 0.7373493975903614 0.5328730817537433 0.19951535778366453 0.20718069452619187
[[6380 1051]
 [1347  352]]


# N = 5

In [76]:
# xy_df_sub5 = pd.read_csv('data/xy_df/xy_df_sub_5_combined.csv')
# xy_df_sub5 = xy_df_sub5.drop([ 'latitude', 'longitude', 'Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month', 'End Day', 'label', 'Unnamed: 0'], axis=1)
# x = xy_df_sub5.drop(xy_df_sub5.filter(regex='target').columns, axis=1)#drop target col
# x = x.select_dtypes(['number'])#drop index col
# y = xy_df_sub5.filter(regex='target') #filter to cols containing target

# column_index = x.columns.get_loc('year')
# x = x.iloc[:, column_index+1:]
# x_train5, x_test5, y_train5, y_test5 = train_test_split(x, y, test_size=0.3, random_state=42)

# print(x_train5.shape, x_test5.shape, y_train5.shape, y_test5.shape)

xy_df_sub5 = pd.read_csv('data/xy_df_renamed/xy_df_sub5_full.csv')
x = xy_df_sub5.drop(xy_df_sub5.filter(regex='target').columns, axis=1)  # Drop target columns
# x = x.drop(x.filter(regex='flood'), axis=1)
x = x.select_dtypes(['number'])  # Keep only numerical columns
y = xy_df_sub5.filter(regex='target')  # Keep only target columns

column_index = x.columns.get_loc('year')
x = x.iloc[:, column_index+1:]

# Get the 'year' column
years = xy_df_sub5['year']

# Determine the split based on years
min_year, max_year = years.min(), years.max()
unique_years = np.sort(years.unique())
split_year = unique_years[int(len(unique_years) * 0.8)]  # 80% split point

# Split data chronologically
train_mask = years <= split_year
test_mask = years > split_year

x_train5, x_test5 = x[train_mask], x[test_mask]
y_train5, y_test5 = y[train_mask], y[test_mask]

print(x_train5.shape, x_test5.shape, y_train5.shape, y_test5.shape)


(39840, 539) (9130, 539) (39840, 1) (9130, 1)


In [77]:
results5 = {}

In [78]:
baseline(y_test5, results5)

Dumb baseline AUC 0.5
maximum f1 score, thres 0.4040080945231412 0.4
auc, f1, accu, accu_bl, precision, recall=  0.5 0.4040080945231412 0.6778751369112815 0.5 0.3221248630887185 0.0
[[6189    0]
 [2941    0]]


In [79]:
stats_only(x_train5, y_train5, x_test5, y_test5, results5)

running xgb...
[10, 20, 65]
Train AUC:  0.6753378146601917
{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.46235076637902717 0.7
auc, f1, accu, accu_bl, precision, recall=  0.4876440300103577 0.46235076637902717 0.4704271631982475 0.488670656481108 0.3173139558307673 0.5399523971438286
[[2707 3482]
 [1353 1588]]


In [80]:
nlp_only(x_train5, y_train5, x_test5, y_test5, results5)

running xgb...
[10, 20, 65]
Train AUC:  0.6705521922326225
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 65}
maximum f1 score, thres 0.2463803121376415 0.7
auc, f1, accu, accu_bl, precision, recall=  0.45561618492714673 0.2463803121376415 0.31883899233296825 0.49133082578588577 0.31838389428736386 0.9761985719143148
[[  40 6149]
 [  70 2871]]


In [81]:
weather_only(x_train5, y_train5, x_test5, y_test5, results5)

running xgb...
[10, 20, 65]
Train AUC:  0.7573744170230869
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 20}
maximum f1 score, thres 0.31419849935795013 0.7
auc, f1, accu, accu_bl, precision, recall=  0.507322635189425 0.31419849935795013 0.3502738225629792 0.49444224045590096 0.3197178323719675 0.8996939816388984
[[ 552 5637]
 [ 295 2646]]


In [82]:
weather_nlp(x_train5, y_train5, x_test5, y_test5, results5)

running xgb...
[10, 20, 65]
Train AUC:  0.6705521922326225
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 65}
maximum f1 score, thres 0.2463803121376415 0.7
auc, f1, accu, accu_bl, precision, recall=  0.45561618492714673 0.2463803121376415 0.31883899233296825 0.49133082578588577 0.31838389428736386 0.9761985719143148
[[  40 6149]
 [  70 2871]]


In [83]:
stats_nlp(x_train5, y_train5, x_test5, y_test5, results5)

running xgb...
[10, 20, 65]
Train AUC:  0.6843352974247028
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.45318379367902445 0.7
auc, f1, accu, accu_bl, precision, recall=  0.47688726568383244 0.45318379367902445 0.46341730558598027 0.4748456873804414 0.3118318423120624 0.5069704182250935
[[2740 3449]
 [1450 1491]]


In [84]:
stats_weather(x_train5, y_train5, x_test5, y_test5, results5)

running xgb...
[10, 20, 65]
Train AUC:  0.756860532629901
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.45770787088136816 0.7
auc, f1, accu, accu_bl, precision, recall=  0.515506254337128 0.45770787088136816 0.45870755750273823 0.507685015956346 0.32553614319922475 0.6453587215232914
[[2290 3899]
 [1043 1898]]


In [85]:
all_vars(x_train5, y_train5, x_test5, y_test5, results5)

running xgb...
[10, 20, 65]
Train AUC:  0.7653933577582377
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.44864143042058113 0.7
auc, f1, accu, accu_bl, precision, recall=  0.4934142130285775 0.44864143042058113 0.45312157721796276 0.482062371795305 0.3146152673398642 0.5634138048282897
[[2480 3709]
 [1284 1657]]
